In [1]:
ref='nb9-' #Note to matt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize':16,
    'axes.labelsize':16,
    'axes.xmargin':0.1,
    'axes.ymargin':0.1,
    'legend.fontsize':16,
    'xtick.labelsize' : 16,
    'ytick.labelsize' : 16,
    'lines.markersize': 10,
    'lines.linewidth' : 3,
    'font.size': 16
})


import pickle 
import tempfile
import os


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
import mlflow.tensorflow

RANDOM_STATE=24


# Define MLP Model

In [2]:
def make_mlp(inp_dim, run): #Functional API #Default initializer for all layers is Xavier (aka glorot_uniform)
    inputs = keras.Input(shape=(inp_dim,), name='input') 
    x = Dense(run['nodes'][0], activation=run['act'][0])(inputs)
    x = Dropout(run['dropout'][0])(x)
    for i in range(int(len(run['nodes'])-1)):
        x = Dense(run['nodes'][i+1], activation=run['act'][i+1])(x)
        x=Dropout(run['dropout'][i+1])(x)
    outputs = Dense(1, activation='linear')(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name='model')
    return model

# Import Data

In [3]:
def make_data(data_key):
    if data_key=='xyz_raw':
        data_file_path="../data/processed/mlp_dataset_raw_xyz.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:300].values
        y=dataset.iloc[:,300].values
    elif data_key=='xyz_interp':
        data_file_path="../data/processed/mlp_dataset_interp_xyz.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:300].values
        y=dataset.iloc[:,300].values
    elif data_key=='rms_raw':
        data_file_path="../data/processed/mlp_dataset_raw_rms.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:100].values
        y=dataset.iloc[:,100].values
    elif data_key == 'rms_interp':
        data_file_path="../data/processed/mlp_dataset_interp_rms.csv"
        dataset=pd.read_csv(data_file_path,index_col=False )
        X=dataset.iloc[:,0:100].values
        y=dataset.iloc[:,100].values
    #Split training/testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
    return X_train, X_test,y_train, y_test, data_file_path

# Import Hyperparameters

In [4]:
# Import Hyperparameters
from mlp_hyperparams_9 import grid

No. of combinations: 54


# Train and Test

In [5]:
mlflow.set_experiment("Gait Speed Prediction")

for i,run in enumerate(grid):
    with mlflow.start_run(run_name=str(run['id'])):
        mlflow.set_tag('Model', run['model'])
        mlflow.set_tag('Dataset', run['dataset'])
        mlflow.set_tag('Data splitting Random State', RANDOM_STATE)
        X_train, X_test,y_train, y_test, data_file_path=make_data(run['dataset'])
        mlflow.set_tag('Data File Path', data_file_path)
        # mlflow.log_artifact(data_file_path) #Uncomment this if dataset needs ot be saved along with model
        
        scaler_X=run['scaler_X']
        sX_train=scaler_X.fit_transform(X_train)
        sX_test=scaler_X.transform(X_test)
        with tempfile.TemporaryDirectory() as tmpdir:
            pickle.dump(scaler_X, open(os.path.join(tmpdir,'scaler_X.pkl'), 'wb'))
            mlflow.log_artifact(os.path.join(tmpdir,'scaler_X.pkl'))
        mlflow.set_tag('Scaler_X', str(scaler_X.get_params()))
        
        scaler_y=run['scaler_y']
        if scaler_y == 'none':
            sy_train=y_train
            sy_test=y_test
            mlflow.set_tag('Scaler_y','No scale is applied to traget (i.e. speed)')
        else:
            sy_train=scaler_y.fit_transform(y_train.reshape(-1, 1))
            sy_test=scaler_y.transform(y_test.reshape(-1, 1))
            with tempfile.TemporaryDirectory() as tmpdir:
                pickle.dump(scaler_y, open(os.path.join(tmpdir,'scaler_y.pkl'), 'wb'))
                mlflow.log_artifact(os.path.join(tmpdir,'scaler_y.pkl'))
            mlflow.set_tag('Scaler_X', str(scaler_y.get_params()))

        model=make_mlp(sX_train.shape[1], run)
        model.compile(loss=run['loss'] , optimizer="adam", metrics=run['metrics'])
        mlflow.tensorflow.autolog() #This will log params passed in model.fit
        history = model.fit(sX_train,sy_train, batch_size=run['batch_size'], epochs=run['epochs'], verbose=1, validation_split=0.1)
        # Testing
        
        score = model.evaluate(sX_test, sy_test,  verbose=1)
        mlflow.log_metrics({'test_loss': score[0], 'test_RMSE': np.sqrt(score[1])})
    

2022-01-12 16:07:13.406316: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-12 16:07:13.413872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-01-12 16:07:13.413903: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-12 16:07:13.414746: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

Epoch 1/20
919/919 [==============================] - 4s 4ms/step - loss: 0.6518 - mean_squared_error: 0.6518 - val_loss: 0.5868 - val_mean_squared_error: 0.5868
Epoch 2/20
919/919 [==============================] - 3s 3ms/step - loss: 0.5748 - mean_squared_error: 0.5748 - val_loss: 0.5840 - val_mean_squared_error: 0.5840
Epoch 3/20
919/919 [==============================] - 3s 3ms/step - loss: 0.5508 - mean_squared_error: 0.5508 - val_loss: 0.5817 - val_mean_squared_error: 0.5817
Epoch 4/20
919/919 [==============================] - 3s 3ms/step - loss: 0.5406 - mean_squared_error: 0.5406 - val_loss: 0.6168 - val_mean_squared_error: 0.6168
Epoch 5/20
919/919 [==============================] - 2s 3ms/step - loss: 0.5298 - mean_squared_error: 0.5298 - val_loss: 0.5985 - val_mean_squared_error: 0.5985
Epoch 6/20
919/919 [==============================] - 3s 3ms/step - loss: 0.5190 - mean_squared_error: 0.5190 - val_loss: 0.5696 - val_mean_squared_error: 0.5696
Epoch 7/20
919/919 [========

2022-01-12 16:08:07.450734: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpkf4qrt2b/model/data/model/assets
128/128 [==============================] - 0s 1ms/step - loss: 0.5693 - mean_squared_error: 0.5693
Epoch 1/20
919/919 [==============================] - 4s 4ms/step - loss: 0.8440 - mean_squared_error: 0.8440 - val_loss: 0.7608 - val_mean_squared_error: 0.7608
Epoch 2/20
919/919 [==============================] - 3s 3ms/step - loss: 0.7522 - mean_squared_error: 0.7522 - val_loss: 0.8191 - val_mean_squared_error: 0.8191
Epoch 3/20
919/919 [==============================] - 3s 3ms/step - loss: 0.7098 - mean_squared_error: 0.7098 - val_loss: 0.7691 - val_mean_squared_error: 0.7691
Epoch 4/20
919/919 [==============================] - 3s 3ms/step - loss: 0.6581 - mean_squared_error: 0.6581 - val_loss: 0.7019 - val_mean_squared_error: 0.7019
Epoch 5/20
919/919 [==============================] - 3s 3ms/step - loss: 0.6239 - mean_squared_error: 0.6239 - val_loss: 0.6414 - val_mean_squared_error: 0.6414
Epoch 6/20
919/

In [6]:
# #Predicting based on selected run (to be used in production)

# #Retrieve Scaler
# client = mlflow.tracking.MlflowClient() 
# local_dir = "/tmp/artifact_downloads" 
# if not os.path.exists(local_dir): 
# 	os.mkdir(local_dir) 

# local_path = client.download_artifacts('3329b6cce2fd426cad7194f9da526103', '', local_dir) 
# file = open('/tmp/artifact_downloads/scaler.pkl', 'rb')
# scaler=pickle.load(file)

# #Retrieve Model
# logged_model = 'runs:/3329b6cce2fd426cad7194f9da526103/model'
# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# #Predict 
# sX_test=scaler.transform(X_test)
# loaded_model.predict(sX_test)